In [7]:
# dicionário para inferir o formato correto no dataset

schema = {
    "_id": "Int64",
    "DatGeracaoConjuntoDados": "str",
    "AnmPeriodoReferencia": "str",
    "NumCPFJDistribuidora": "Int64",
    "SigAgente": "str",
    "NomAgente": "str",
    "CodClasseConsumo": "Int64",
    "DscClasseConsumo": "str",
    "CodSubGrupoTarifario": "Int64",
    "DscSubGrupoTarifario": "str",
    "codUFibge": "Int64",
    "SigUF": "str",
    "codRegiao": "str",
    "NomRegiao": "str",
    "CodMunicipioIbge": "Int64",
    "NomMunicipio": "str",
    "CodCEP": "str",
    "SigTipoConsumidor": "str",
    "NumCPFCNPJ": "str",
    "NomeTitularEmpreendimento": "str",
    "CodEmpreendimento": "str",
    "DthAtualizaCadastralEmpreend": "str",
    "SigModalidadeEmpreendimento": "str",
    "DscModalidadeHabilitado": "str",
    "QtdUCRecebeCredito": "Int64",
    "SigTipoGeracao": "str",
    "DscFonteGeracao": "str",
    "DscPorte": "str",
    "MdaPotenciaInstaladaKW": "str",
    "NumCoordNEmpreendimento": "str",
    "NumCoordEEmpreendimento": "str",
    "NomSubEstacao": "str",
    "NumCoordESub": "str",
    "NumCoordNSub": "str",
  }

# colunas que são data
dates = ["DatGeracaoConjuntoDados", "DthAtualizaCadastralEmpreend"]

In [8]:
import pandas as pd
import re

df = pd.read_csv('dados_aneel.csv', dtype=schema, parse_dates=dates)

df.head()

,_id,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,...,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
0,1,2023-05-18,05/2023,3.305020e+13,CPFL-PAULISTA,COMPANHIA PAULISTA DE FORÇA E LUZ,1,Residencial,9,B1,...,1,UFV,Radiação solar,Microgeracao,"3,30",NaN,NaN,NaN,NaN,NaN
1,2,2023-05-18,05/2023,4.895728e+12,Equatorial PA,EQUATORIAL PARÁ DISTRIBUIDORA DE ENERGIA S.A.,1,Residencial,9,B1,...,1,UFV,Radiação solar,Microgeracao,"3,60",NaN,NaN,NaN,NaN,NaN
2,3,2023-05-18,05/2023,4.065033e+12,ELETROACRE,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,2,Comercial,11,B3,...,1,UFV,Radiação solar,Microgeracao,"32,50","-67,84","-9,98",NaN,NaN,NaN
3,4,2023-05-18,05/2023,4.065033e+12,ELETROACRE,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,1,Residencial,9,B1,...,1,UFV,Radiação solar,Microgeracao,"4,00","-70,78","-8,15",NaN,NaN,NaN
4,5,2023-05-18,05/2023,4.065033e+12,ELETROACRE,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,2,Comercial,11,B3,...,1,UFV,Radiação solar,Microgeracao,"2,00",NaN,NaN,NaN,NaN,NaN


In [9]:
# checar os valores null, e se é possível inferir valores

df.isna().sum()

_id                                   0
DatGeracaoConjuntoDados               0
AnmPeriodoReferencia                  0
NumCNPJDistribuidora                135
SigAgente                           135
NomAgente                           135
CodClasseConsumo                      0
DscClasseConsumo                      0
CodSubGrupoTarifario                  0
DscSubGrupoTarifario                  0
codUFibge                          3955
SigUF                               170
codRegiao                          3955
NomRegiao                             0
CodMunicipioIbge                    170
NomMunicipio                        170
CodCEP                                2
SigTipoConsumidor                     2
NumCPFCNPJ                            2
NomeTitularEmpreendimento             2
CodEmpreendimento                     2
DthAtualizaCadastralEmpreend          0
SigModalidadeEmpreendimento           0
DscModalidadeHabilitado             180
QtdUCRecebeCredito                    0


In [10]:
# visto que possui valores nulos, é possível tentar inferir alguns valores em colunas onde
# só há um valor possível para cada feature, no caso é possível inferir o NomMunicipio e CodMunicipio
# pelo CEP, apesar de o CEP estar incompleto, todos possuem pelo menos os 5 primeiros números, o que aumenta
# a precisão do município (fonte: https://www.significados.com.br/cep/)
# outro dado que é possível inferir é o codigo do estado, dado que muitos projetos estão sem o codigo
# mas possuem a sigla do estado, o codigo da regiao possui mais de um por estado, então não é possível inferir
# e por ultimo inferir a descricao da modalidade do projeto, dado que 180 estao sem descrição, mas possuem a sigla
# por ser um número baixo de dados vazios, provavelmente a análise seria efetiva mesmo sem essa etapa

cep_dict_mun_nome = df.set_index('CodCEP')['NomMunicipio'].dropna().to_dict()
cep_dict_mun_cod = df.set_index('CodCEP')['CodMunicipioIbge'].dropna().to_dict()
uf_dict_cod = df.set_index('SigUF')['codUFibge'].dropna().to_dict()
modalidade_dict = df.set_index('SigModalidadeEmpreendimento')['DscModalidadeHabilitado'].dropna().to_dict()

In [11]:
df['NomMunicipio'] = df['NomMunicipio'].fillna(df['CodCEP'].map(cep_dict_mun_nome))
df['CodMunicipioIbge'] = df['CodMunicipioIbge'].fillna(df['CodCEP'].map(cep_dict_mun_cod))
df['codUFibge'] = df['codUFibge'].fillna(df['SigUF'].map(uf_dict_cod))
df['DscModalidadeHabilitado'] = df['DscModalidadeHabilitado'].fillna(df['SigModalidadeEmpreendimento'].map(uf_dict_cod))

df.isna().sum()

_id                                   0
DatGeracaoConjuntoDados               0
AnmPeriodoReferencia                  0
NumCNPJDistribuidora                135
SigAgente                           135
NomAgente                           135
CodClasseConsumo                      0
DscClasseConsumo                      0
CodSubGrupoTarifario                  0
DscSubGrupoTarifario                  0
codUFibge                           170
SigUF                               170
codRegiao                          3955
NomRegiao                             0
CodMunicipioIbge                    129
NomMunicipio                        129
CodCEP                                2
SigTipoConsumidor                     2
NumCPFCNPJ                            2
NomeTitularEmpreendimento             2
CodEmpreendimento                     2
DthAtualizaCadastralEmpreend          0
SigModalidadeEmpreendimento           0
DscModalidadeHabilitado             180
QtdUCRecebeCredito                    0


In [12]:
# altera o CNPJ, que pandas interpreta como float, para string
# todo CNPJ tem 14 digitos, se o pandas converteu algum CNPJ com 0 no começo, irá se perder
# usando zfill ele adiciona o 0 no começo se não houver 14 digítos

df["NumCNPJDistribuidora"] = df["NumCNPJDistribuidora"].astype(str).str.replace('\.0$', '', regex=True)
df['NumCNPJDistribuidora'] = df['NumCNPJDistribuidora'].str.zfill(14)

# converte os floats de formato br para formato us, para o bigquery não distorcer o número

df["MdaPotenciaInstaladaKW"] = df["MdaPotenciaInstaladaKW"].astype(str).str.replace(',', '.').astype(float)
df["NumCoordNEmpreendimento"] = df["NumCoordNEmpreendimento"].astype(str).str.replace(',', '.').astype(float)
df["NumCoordEEmpreendimento"] = df["NumCoordEEmpreendimento"].astype(str).str.replace(',', '.').astype(float)
df["NumCoordESub"] = df["NumCoordESub"].astype(str).str.replace(',', '.').astype(float)
df["NumCoordNSub"] = df["NumCoordNSub"].astype(str).str.replace(',', '.').astype(float)

df.head()

,_id,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,...,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
0,1,2023-05-18,05/2023,33050196000188,CPFL-PAULISTA,COMPANHIA PAULISTA DE FORÇA E LUZ,1,Residencial,9,B1,...,1,UFV,Radiação solar,Microgeracao,3.3,NaN,NaN,NaN,NaN,NaN
1,2,2023-05-18,05/2023,04895728000180,Equatorial PA,EQUATORIAL PARÁ DISTRIBUIDORA DE ENERGIA S.A.,1,Residencial,9,B1,...,1,UFV,Radiação solar,Microgeracao,3.6,NaN,NaN,NaN,NaN,NaN
2,3,2023-05-18,05/2023,04065033000170,ELETROACRE,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,2,Comercial,11,B3,...,1,UFV,Radiação solar,Microgeracao,32.5,-67.84,-9.98,NaN,NaN,NaN
3,4,2023-05-18,05/2023,04065033000170,ELETROACRE,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,1,Residencial,9,B1,...,1,UFV,Radiação solar,Microgeracao,4.0,-70.78,-8.15,NaN,NaN,NaN
4,5,2023-05-18,05/2023,04065033000170,ELETROACRE,ENERGISA ACRE - DISTRIBUIDORA DE ENERGIA S.A,2,Comercial,11,B3,...,1,UFV,Radiação solar,Microgeracao,2.0,NaN,NaN,NaN,NaN,NaN


In [13]:
# checar se os dtypes estão corretos

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923800 entries, 0 to 1923799
Data columns (total 34 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   _id                           Int64         
 1   DatGeracaoConjuntoDados       datetime64[ns]
 2   AnmPeriodoReferencia          object        
 3   NumCNPJDistribuidora          object        
 4   SigAgente                     object        
 5   NomAgente                     object        
 6   CodClasseConsumo              Int64         
 7   DscClasseConsumo              object        
 8   CodSubGrupoTarifario          Int64         
 9   DscSubGrupoTarifario          object        
 10  codUFibge                     Int64         
 11  SigUF                         object        
 12  codRegiao                     object        
 13  NomRegiao                     object        
 14  CodMunicipioIbge              Int64         
 15  NomMunicipio                  ob

In [14]:
# checa se tem 110 CNPJ's (conforme base original) por conta da transformação

df['NumCNPJDistribuidora'].drop_duplicates()

0          33050196000188
1          04895728000180
2          04065033000170
4617       12272084000100
4750       15139629000194
                ...      
1583049    60196987000193
1588140    55188502000180
1588501    52548732000114
1590665    44560381000139
1895976    25086034000171
Name: NumCNPJDistribuidora, Length: 110, dtype: object

In [15]:
df.to_parquet('dados_aneel_clean.parquet')